In [4]:
import sqlite3
import csv

DB_PATH = r"C:\Users\nvbma\my_testdb.db"
OUTPUT_FILE = "output_sql.csv"


def connect_db():
    try:
        return sqlite3.connect(DB_PATH)
    except Exception as e:
        print("Database connection failed:", e)
        raise


def run_query(conn):
    query = """
    SELECT 
        CASE 
            WHEN c.age BETWEEN 18 AND 25 THEN '18-25'
            WHEN c.age BETWEEN 26 AND 35 THEN '26-35'
            WHEN c.age BETWEEN 36 AND 50 THEN '36-50'
            ELSE '50+'
        END AS Age_Group,
        i.item_name AS Item,
        SUM(o.quantity) AS Total_Quantity
    FROM Customer c
    JOIN Sales s ON c.customer_id = s.customer_id
    JOIN Orders o ON s.sales_id = o.sales_id
    JOIN Items i ON o.item_id = i.item_id
    WHERE o.quantity IS NOT NULL
    GROUP BY 
        Age_Group,
        i.item_name
    HAVING SUM(o.quantity) > 0
    ORDER BY Age_Group, i.item_name;
    """
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        return cursor.fetchall()
    except Exception as e:
        print("Query execution failed:", e)
        raise


def write_csv(data):
    try:
        with open(OUTPUT_FILE, "w", newline="") as f:
            writer = csv.writer(f, delimiter=';')
            writer.writerow(["Age_Group", "Item", "Total_Quantity"])
            for row in data:
                writer.writerow(row)
    except Exception as e:
        print("CSV writing failed:", e)
        raise


def main():
    conn = None
    try:
        conn = connect_db()
        data = run_query(conn)
        write_csv(data)
        print("SQL solution completed successfully.")
    finally:
        if conn:
            conn.close()


if __name__ == "__main__":
    main()

SQL solution completed successfully.


In [6]:
import sqlite3
import pandas as pd

DB_PATH = r"C:\Users\nvbma\my_testdb.db"
OUTPUT_FILE = "output_pandas.csv"


def validate_tables(conn):
    required_tables = {"Customer", "Sales", "Orders", "Items"}
    existing = pd.read_sql(
        "SELECT name FROM sqlite_master WHERE type='table';", conn
    )["name"].tolist()

    missing = required_tables - set(existing)
    if missing:
        raise Exception(f"Missing tables: {missing}")


def main():
    conn = None
    try:
        conn = sqlite3.connect(DB_PATH)

        # Validate required tables exist
        validate_tables(conn)

        # Read tables
        customers = pd.read_sql("SELECT * FROM Customer", conn)
        sales = pd.read_sql("SELECT * FROM Sales", conn)
        orders = pd.read_sql("SELECT * FROM Orders", conn)
        items = pd.read_sql("SELECT * FROM Items", conn)

        # Merge tables (Header → Detail relationship)
        df = (
            customers
            .merge(sales, on="customer_id", how="inner")
            .merge(orders, on="sales_id", how="inner")
            .merge(items, on="item_id", how="inner")
        )

        # Ensure quantity is numeric
        df["quantity"] = pd.to_numeric(df["quantity"], errors="coerce")

        # Apply business rules
        df = df[
            (df["age"].between(18, 35)) &        # Age filter
            (df["quantity"].notna())             # Ignore NULL purchases
        ]

        # Group and aggregate safely
        result = (
            df.groupby(["customer_id", "age", "item_name"], as_index=False)["quantity"]
              .sum()
        )

        # Remove zero totals
        result = result[result["quantity"] > 0]

        # Rename columns for output
        result.rename(columns={
            "customer_id": "Customer",
            "age": "Age",
            "item_name": "Item",
            "quantity": "Quantity"
        }, inplace=True)

        # Export
        result.to_csv(OUTPUT_FILE, sep=";", index=False)

        print("Pandas solution completed successfully.")

    except Exception as e:
        print("Error:", e)

    finally:
        if conn:
            conn.close()


if __name__ == "__main__":
    main()

Pandas solution completed successfully.
